In [ ]:
#!/usr/bin/env python

import networkx as nx
import random
import pygraphviz as pygv
import gurobi as gb
import json
import csv
from community import community_louvain
from IPython.display import Image, display

def DrawSol (G, class_dict, filename):
    global n_micros

    DrawG = pygv.AGraph(directed=True, strict='true', splines='true')

    colors = ['green', 'blue', 'red', 'orange', 'yellow', 'purple', 'black', 'gray', 'cyan', 'brown', 'hotpink', 'navy', 'darkgreen', 'chocolate', 'deeppink', 'firebrick', 'gold', 'orchid', 'gold4']
    
    for i in G.nodes():
        node = G.nodes[i]
        if node['type'] == 'Entity':
            shape='hexagon'
        else:
            shape='circle'

        color = 'black'
        for k in class_dict:
            if (node['type'] == 'Entity' and node['name'] in class_dict[k]) or (node['type'] != 'Entity' and node['class_name'] in class_dict[k]):
                color = colors[k]

        DrawG.add_node (i, color=color, shape=shape, width=0.1, fontsize=9, label=G.nodes[i]['name'])

    for i in G.edges():
        edge_color = 'black' if G[i[0]][i[1]]['rel_type'] in ['Calls','Persists', 'References'] else 'gray'
        DrawG.add_edge(i[0], i[1], color=edge_color, label=G[i[0]][i[1]]['rel_type'], fontsize='8')
        
    DrawG.layout(prog='dot')
    DrawG.draw(filename)

### _Configuration!_

Keep the project list up-to-date and select the project to be analyzed

In [ ]:
project = 'spring-petclinic' # <-- Set this variable!

with open('projects.json', 'r') as projects:
    project_found = False
    for data in json.load(projects):
        if data['name'] == project:
            analysis_results_basedir = data['analysis_results_basedir']
            project_found = True
    if not project_found:
        print('ERROR: project ' + project + ' does not appear in project.json')

### _User input required!_

Set the weight of the edges

In [ ]:
# Weighting edges:
w = dict()
w['Calls']=0.8
w['Persists']=1
w['Uses']=0.6
w['References']=0.2
w['Extends']=0

In [ ]:
# Set output names

graph_in = analysis_results_basedir + project + '_graph.gml'
graph_image_out = analysis_results_basedir + project + '_sol_comps.png'

In [ ]:
# Import graph

G = nx.read_graphml(graph_in)

In [ ]:
# Add edge weights

for (i, j) in G.edges():
    G[i][j]['w'] = w[G[i][j]['rel_type']]

In [ ]:
baseline = {
    0: ['OwnerController', 'Owner', 'OwnerRepository', 'PetController', 'Pet', 'PetRepository', 'PetType'],
    1: ['VisitController', 'Visit', 'VisitRepository'],
    2: ['VetController', 'Specialty', 'Vet', 'VetRepository']
}

kamimura = {
    0: ['OwnerController', 'Owner', 'OwnerRepository', 'PetController', 'Pet', 'PetType', 'BaseEntity', 'NamedEntity'],
    1: ['VisitController', 'VisitRepository', 'PetRepository', 'Pet', 'Visit'],
    2: ['VetController', 'Vet', 'VetRepository', 'Vets', 'Specialty']
}

In [ ]:
watching = baseline # <-- Set this variable

class_dict = watching

## Result Analysis

In [ ]:
# Compute coupling and cohesion:

coupling = 0
mcalls = []
references = []
uses = []
persist = []

inside_w = dict()
outside_w = dict()

for i,j in G.edges():
#    same_micro = False

    nodei = G.nodes[i]
    nodej = G.nodes[j]

    for k in class_dict:
        if (nodei['type'] == 'Entity' and nodei['name'] in class_dict[k]) or (nodei['type'] != 'Entity' and nodei['class_name'] in class_dict[k]):
            partitioni = k 
        if (nodej['type'] == 'Entity' and nodej['name'] in class_dict[k]) or (nodej['type'] != 'Entity' and nodej['class_name'] in class_dict[k]):
            partitionj = k 

    outside_w[partitioni] = outside_w.get(partitioni, 0) + G[i][j]['w']
            
    if partitioni != partitionj:
        coupling += w[G[i][j]['rel_type']]
        if G[i][j]['rel_type'] == 'Calls':
            mcalls.append(f'{nodei["name"]}({nodei["class_name"]}) Calls {nodej["name"]}({nodej["class_name"]})')
        elif G[i][j]['rel_type'] == 'References':
            references.append(f'{nodei["name"]} References {nodej["name"]}')
        elif G[i][j]['rel_type'] == 'Uses':
            uses.append(f'{nodei["name"]}({nodei["class_name"]}) Uses {nodej["name"]}')
        elif G[i][j]['rel_type'] == 'Persists':
            persist.append(f'{nodei["name"]}({nodei["class_name"]}) Persists {nodej["name"]}')
    else:
        inside_w[partitioni] = inside_w.get(partitioni, 0) + G[i][j]['w']

cohesion_dict = dict()
for k in range(len(class_dict.values())):
    cohesion_dict[k] = inside_w[k] / outside_w[k]

cohesion = sum(cohesion_dict.values())/len(class_dict.values())

In [ ]:
# Hints: prima identificare i cluster di entità guardando alle dipendenze tra metodi ed entità. Poi usare le metriche per ottenere la suddivizione del sistema.

analysis_output = ""

analysis_output += f'Found {len(class_dict)} microservices\n'

analysis_output += f'Coupling value: {coupling} ({coupling/len(class_dict.values())} avg.)\n'
analysis_output += f'Cohesion value: {cohesion}\n'

for k in range(len(class_dict)):
    analysis_output += f'\nEntities in Microservice {str(k+1)}:\n'
    for i in G.nodes():
        if G.nodes[i]['type'] == 'Entity' and G.nodes[i]['name'] in class_dict[k]:
            analysis_output += f'- {G.nodes[i]["name"]}\n'

#for k in range(len(class_dict)):
    #analysis_output += f'\n\nMethods in Microservice {str(k)}:\n'
    #for i in G.nodes():
        #if G.nodes[i]['type'] == 'Method' and G.nodes[i]['class_name'] in class_dict[k]:
            #analysis_output += f'- {G.nodes[i]["name"]} ({G.nodes[i]["class_name"]})\n'

analysis_output += f'\n\n# of method calls crossing microservices: {len(mcalls)}'
analysis_output += f'\n# of entity references crossing microservices: {len(references)}'
analysis_output += f'\n# of entity usages crossing microservices: {len(uses)}'
analysis_output += f'\n# of methods persisting entities of other microservices: {len(persist)}'

print(analysis_output)

In [ ]:
DrawSol(G, class_dict, graph_image_out)

display(Image(filename = graph_image_out))